# fz-Scale Plugin Example Usage

This notebook demonstrates how to use the fz-Scale plugin for SCALE calculations.

The examples cover:
1. Installing dependencies (fz framework and fz-Scale plugin)
2. Downloading example input files
3. Parsing input files to identify variables
4. Compiling input files with specific parameter values
5. Running SCALE KENO criticality calculations
6. Performing parametric studies

**Note:** The execution examples require SCALE to be installed. If SCALE is not available, those examples will fail but demonstrate the plugin structure.

## Setup: Install Dependencies

First, install the Funz framework and the fz-Scale plugin.

In [ ]:
# Install fz framework from GitHub
%pip install -q git+https://github.com/Funz/fz.git

# Install fz-Scale plugin
import fz
fz.install("Scale")

print("✓ Dependencies installed successfully!")

## Setup: Download Example Files

Download the example SCALE input files from the GitHub repository.

In [ ]:
import os
import urllib.request

# Create directories for examples
os.makedirs("Scale-keno", exist_ok=True)
os.makedirs("Scale-shift", exist_ok=True)

# Base URL for raw GitHub content
base_url = "https://raw.githubusercontent.com/Funz/fz-Scale/main/examples"

# Download example files
files_to_download = [
    ("Scale-keno/godiva.inp", f"{base_url}/Scale-keno/godiva.inp"),
    ("Scale-shift/godiva.inp", f"{base_url}/Scale-shift/godiva.inp"),
]

print("Downloading example files...")
for local_path, url in files_to_download:
    try:
        urllib.request.urlretrieve(url, local_path)
        print(f"  ✓ Downloaded: {local_path}")
    except Exception as e:
        print(f"  ✗ Failed to download {local_path}: {e}")

print("\n✓ Example files ready!")

## Import Required Libraries

In [ ]:
import fz
import os
import tempfile

print(f"fz version: {fz.__version__ if hasattr(fz, '__version__') else 'unknown'}")

## Example 1: Parse Input File for Variables

Use `fz.fzi()` to identify variables in a SCALE input file.

In [ ]:
# Parse the input file to find variables
variables = fz.fzi(
    "Scale-keno/godiva.inp",
    "Scale-keno"
)

print(f"Variables found in godiva.inp: {list(variables.keys())}")

## Example 2: Compile Input Files

Use `fz.fzc()` to compile input files with specific parameter values. This substitutes variables with actual values.

In [ ]:
# Create temporary directory for compiled files
with tempfile.TemporaryDirectory() as tmpdir:
    # Compile input with specific radius
    fz.fzc(
        "Scale-keno/godiva.inp",
        {"r": 8.741},
        "Scale-keno",
        output_dir=tmpdir
    )
    
    # Show compiled file
    compiled_file = os.path.join(tmpdir, "godiva.inp")
    if os.path.exists(compiled_file):
        print("Compiled file content (r=8.741):")
        print("=" * 70)
        with open(compiled_file, 'r') as f:
            for i, line in enumerate(f, 1):
                if i <= 15:  # Show first 15 lines
                    print(f"{line.rstrip()}")

## Example 3: SCALE KENO - Godiva Critical Sphere

Run a KENO parametric study with varying sphere radius to find the critical configuration.

The Godiva sphere is a classic nuclear criticality benchmark. We'll test radii around the critical radius of 8.741 cm.

In [ ]:
# Define parameter values
# Testing radii around the critical radius of 8.741 cm
input_variables = {
    "r": [8.5, 8.6, 8.7, 8.741, 8.8, 8.9, 9.0]
}

try:
    # Run parametric calculation
    results = fz.fzr(
        input_path="Scale-keno/godiva.inp",
        input_variables=input_variables,
        model="Scale-keno",
        calculators="Localhost_Scale-keno",
        results_dir="results/godiva_study"
    )
    
    # Display results
    print("\nResults:")
    print("=" * 70)
    print(results[['r', 'mean_keff', 'sigma_keff', 'status']])
    
    # Find critical radius (k-eff closest to 1.0)
    results['delta_k'] = abs(results['mean_keff'] - 1.0)
    critical = results.loc[results['delta_k'].idxmin()]
    print(f"\nClosest to critical: r={critical['r']} cm, k-eff={critical['mean_keff']}")
    
except Exception as e:
    print(f"\nError running calculation: {e}")
    print("Note: This requires SCALE to be installed.")

## Example 4: Advanced Parametric Study with Visualization

This example demonstrates a more comprehensive parametric study with:
- Fine-grained parameter sweep
- Result visualization
- Statistical analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define a fine-grained parameter sweep
# Explore radii from 8.0 cm to 9.5 cm in 0.1 cm increments
radii = np.arange(8.0, 9.6, 0.1)
input_variables = {"r": radii.tolist()}

print(f"Running parametric study with {len(radii)} radius values...")
print(f"Radius range: {radii[0]:.1f} to {radii[-1]:.1f} cm")

try:
    # Run parametric calculation
    results = fz.fzr(
        input_path="Scale-keno/godiva.inp",
        input_variables=input_variables,
        model="Scale-keno",
        calculators="Localhost_Scale-keno",
        results_dir="results/godiva_parametric_study"
    )
    
    # Filter successful results
    successful = results[results['status'] == 'success'].copy()
    
    # Statistical summary
    print("\n" + "=" * 70)
    print("PARAMETRIC STUDY RESULTS")
    print("=" * 70)
    print(f"\nTotal runs: {len(results)}")
    print(f"Successful: {len(successful)}")
    print(f"Failed: {len(results) - len(successful)}")
    
    if len(successful) > 0:
        # Find critical configuration
        successful['delta_k'] = abs(successful['mean_keff'] - 1.0)
        critical = successful.loc[successful['delta_k'].idxmin()]
        
        print(f"\nCritical configuration:")
        print(f"  Radius: {critical['r']:.3f} cm")
        print(f"  k-eff: {critical['mean_keff']:.5f} ± {critical['sigma_keff']:.5f}")
        
        # Display sample results
        print("\nSample results:")
        print(successful[['r', 'mean_keff', 'sigma_keff']].head(10))
        
        # Visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: k-eff vs radius
        ax1.errorbar(
            successful['r'], 
            successful['mean_keff'], 
            yerr=successful['sigma_keff'],
            fmt='o-', 
            capsize=3, 
            label='KENO results'
        )
        ax1.axhline(y=1.0, color='r', linestyle='--', label='Critical (k-eff = 1.0)')
        ax1.axvline(x=critical['r'], color='g', linestyle='--', alpha=0.5, label=f"Critical r = {critical['r']:.3f} cm")
        ax1.set_xlabel('Sphere Radius (cm)', fontsize=12)
        ax1.set_ylabel('k-effective', fontsize=12)
        ax1.set_title('Godiva Sphere Criticality Study', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Plot 2: Reactivity vs radius
        # Reactivity in dollars (assuming β_eff ≈ 0.0064 for HEU)
        beta_eff = 0.0064
        successful['reactivity'] = (successful['mean_keff'] - 1.0) / (beta_eff * successful['mean_keff'])
        
        ax2.plot(successful['r'], successful['reactivity'], 'o-', color='navy')
        ax2.axhline(y=0, color='r', linestyle='--', label='Critical')
        ax2.set_xlabel('Sphere Radius (cm)', fontsize=12)
        ax2.set_ylabel('Reactivity ($)', fontsize=12)
        ax2.set_title('Reactivity vs Radius', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.savefig('results/godiva_parametric_study.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("\nPlot saved to: results/godiva_parametric_study.png")
        
except Exception as e:
    print(f"\nError running parametric study: {e}")
    print("Note: This requires SCALE to be installed and properly configured.")